# 9장. 잠재고객을 파악하기 위한 이미지 인식 테크닉 10

## 81: 이미지 데이터 불러오기

In [1]:
!pip install opencv-python

In [ ]:
import cv2
img = cv2.imread("img/img01.jpg")
height, width = img.shape[:2]
print("이미지 가로: " + str(width))
print("이미지 세로: " + str(height)) 
cv2.imshow("img", img)
cv2.waitKey(0) 

이미지 가로: 1920
이미지 세로: 1440


- cv2: a popular library for image and video processing tasks in Python
- img.shape: returns the dimensions of the image as a tuple (height, width, channels->color channels)
    - img.shape[:2]: (height, width)만 추출
- imshow(): displays the image
    - (note) cv2.imshow() often doesn’t work as expected for displaying images, since it requires a separate GUI window that Jupyter Notebook doesn’t natively support
- waitKey: 몇 초 동안 이미지를 표시할지 밀리초 단위로 지정
    - 인수에 0을 지정할 시, 윈도우를 닫을 때까지 계속해서 보여줌

## 82: 동영상 데이터 불러오기

In [ ]:
import cv2

# 정보 취득
cap = cv2.VideoCapture("mov/mov01.avi")
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
fps = cap.get(cv2.CAP_PROP_FPS)
print("가로: " + str(width))
print("세로: " + str(height))
print("총 프레임수: " + str(count))
print("FPS: " + str(fps))

# 출력
while(cap.isOpened()): #iterates as long as the video file is open&readable
    ret, frame = cap.read() 
    if ret:
        cv2.imshow("frame", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
cap.release()
cv2.destroyAllWindows()

- cap.get(): extract specific properties of the video
- cv2.CAP_PROP_FRAME_WIDTH: 각 video frame의 width
- cv2.CAP_PROP_FRAME_COUNT: num of frames in the video
- cap.read(): reads the next frame of the video
    - ret: boolean. True: frame was read, False: no more frames
    - frame: the actual frame image captured from the video
- cv2.waitKey(1): waits 1 ms between frames -> real-time video display
- cv2.waitKey(1) & 0xFF == ord("q"): checks if the "q" key was pressed

## 83: 동영상을 이미지로 나누고 저장하기

In [ ]:
import cv2
cap = cv2.VideoCapture("mov/mov01.avi")
num = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret:
        cv2.imshow("frame", frame)
        filepath = "snapshot/snapshot_" + str(num) + ".jpg"
        cv2.imwrite(filepath,frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    num = num + 1
cap.release()
cv2.destroyAllWindows()

- cap.release(): 객체가 잡고 있던 비디오 파일 리소스 해제
- cv2.destroyAllWindows(): OpenCV에서 생성한 모든 창 닫음

## 84: 이미지 속에 사람이 어디에 있는지 검출하기

이미지에서 사람을 검출하여 각 사람 영역에 사각형을 그린 후 화면에 표시하고, 결과를 파일로 저장하는 프로그램

In [ ]:
import cv2

# 준비
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
hogParams = {'winStride': (8,8), 'padding': (32, 32), 'scale':1.05, 'hitThreshold':0, 'finalThreshold':5}

# 검출
img = cv2.imread("img/img01.jpg")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
human, r = hog.detectMultiScale(gray, **hogParams)
if (len(human)>0):
    for (x, y, w, h) in human:
        cv2.rectangle(img, (x,y), (x+w, y+h), (255, 255, 255), 3)
        
cv2.namedWindow("img", cv2.WINDOW_NORMAL)
cv2.imshow("img", img)
cv2.imwrite("temp.jpg", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

- HOGDescriptor()
    - Histogram of Oriented Gradients 라는 이미지 특성을 추출하기 위한 기법을 구현한 OpenCV의 클래스
    - 이미지에서 물체(특히 사람)의 윤곽과 같은 특정 패턴을 인식하는 데 유용하게 사용
- hog.setSVMDetector: 사람 검출을 위한 사전 학습된 SVM (Support Vector Machine) 모델선정
- hogParams로 HOG 검출에 필요한 여러 파라메터를 딕셔너리 형태로 정의
- cv2.cvtColor: 그레이스케일로 변환
    - 그레이스케일: 흑백 이미지를 나타내는 방식
    - 각 픽셀의 밝기 정보만을 포함하여 이미지 표현
- hog.detectMultiScale: 사람 검출
    - human: 검출된 객체들의 좌표 (x, y, w, h)가 리스트로 반환
    - 각 좌표는 각각 검출된 사람의 위치와 크기
- cv2.hectangle: 각 검출된 영역에 대해 (x, y, w, h)좌표에 흰색 사각형을 그려 표시

## 85: 이미지 속 사람 얼굴 검출
- 전통적으로 CascadeClassifier 사용

In [ ]:
import cv2

#준비
cascade_file = "haarcascade_frontalface_alt.xml"
cascade = cv2.CascadeClassifier(cascade_file)

#검출
img = cv2.imread("img/img02.jpg")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
face_list = cascade.detectMultiScale(gray, minSize=(50, 50))

#검출한 얼굴 표시하기
for (x, y, w, h) in face_list:
    color = (0, 0, 225)
    pen_w = 3
    cv2.rectangle(img, (x,y), (x+w, y+h), color, thickness = pen_w)
cv2.namedWindow("img",cv2.WINDOW_NORMAL)
cv2.imshow("img",img)
cv2.imwrite("temp.jpg",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

- cv2.CascadeClassifier(): OpenCV에서 Haar Cascade 분류기를 불러오는 클래스
- cascade.detectMultiScale(): 이미지를 탐색하여 얼굴을 검출하는 함수
    - 얼굴이 컴출되면 (x, y, w, h)좌표로 반환
    - minsize=(50, 50): 검출할 얼굴의 최소 크기를 (50, 50)으로 지정 -> 너무 작은 객체는 거름
- cv2.rectangle
    - (x,y): 사각형의 왼쪽 위 모서리
    - (x+w, y+h): 오른쪽 아래 모서리

## 86: 이미지 속 사람의 얼굴이 어느 쪽을 보고 있는지 검출
- dib 라이브러리: 얼굴을 얼굴 랜드마크 (눈, 코, 입, 윤곽)의 68개 특징점으로 표현 가능

In [ ]:
!pip install cmake

In [ ]:
!pip install dlib

In [ ]:
import cv2
import dlib
import math

In [ ]:
# 준비 #
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
detector = dlib.get_frontal_face_detector()

# 검출 #
img = cv2.imread("img/img02.jpg")
dets = detector(img, 1)

for k, d in enumerate(dets):
    shape = predictor(img, d)

    # 얼굴 영역 표시
    color_f = (0, 0, 225)
    color_l_out = (255, 0, 0)
    color_l_in = (0, 255, 0)
    line_w = 3
    circle_r = 3
    fontType = cv2.FONT_HERSHEY_SIMPLEX
    fontSize = 1
    cv2.rectangle(img, (d.left(), d.top()), (d.right(), d.bottom()), color_f, line_w)
    cv2.putText(img, str(k), (d.left(), d.top()), fontType, fontSize, color_f, line_w)

    # 중심을 계산할 사각형 준비
    num_of_points_out = 17
    num_of_points_in = shape.num_parts - num_of_points_out
    gx_out = 0
    gy_out = 0
    gx_in = 0
    gy_in = 0
    for shape_point_count in range(shape.num_parts):
        shape_point = shape.part(shape_point_count)
        #print("얼굴 랜드마크No.{} 좌표 위치: ({},{})".format(shape_point_count, shape_point.x, shape_point.y))
        #얼굴 랜드마크마다 그리기
        if shape_point_count<num_of_points_out:
            cv2.circle(img,(shape_point.x, shape_point.y),circle_r,color_l_out, line_w)
            gx_out = gx_out + shape_point.x/num_of_points_out
            gy_out = gy_out + shape_point.y/num_of_points_out
        else:
            cv2.circle(img,(shape_point.x, shape_point.y),circle_r,color_l_in, line_w)
            gx_in = gx_in + shape_point.x/num_of_points_in
            gy_in = gy_in + shape_point.y/num_of_points_in

    # 중심 위치 표시
    cv2.circle(img,(int(gx_out), int(gy_out)),circle_r,(0,0,255), line_w)
    cv2.circle(img,(int(gx_in), int(gy_in)),circle_r,(0,0,0), line_w)

    # 얼굴 방향 계산
    theta = math.asin(2*(gx_in-gx_out)/(d.right()-d.left()))
    radian = theta*180/math.pi
    print("얼굴 방향:{} (각도:{}도)".format(theta,radian))

    # 얼굴 방향 표시
    if radian<0:
        textPrefix = "   left "
    else:
        textPrefix = "   right "
    textShow = textPrefix + str(round(abs(radian),1)) + " deg."
    cv2.putText(img, textShow, (d.left(), d.top()), fontType, fontSize, color_f, line_w)


# cv2.imshow("img",img)
# cv2.imwrite("temp.jpg",img)
# cv2.waitKey(0)

cv2.namedWindow("img",cv2.WINDOW_NORMAL)
cv2.imshow("img",img)
cv2.imwrite("temp.jpg",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

- enumerate(dets): 여러 얼굴을 검출할 수 있으므로, dets의 각 얼굴 위치와 인덱스를 함께 반환
    - enumerate: 파이썬 내장 함수. 반복문에서 리스트나 다른 iterable 객체의 인덱스와 값을 함께 반환
- predictor(img,d): predictor을 사용해 d 영역에서 얼굴 랜드마크를 찾고, shape 객체로 반환


## 87: 검출한 정보를 종합해서 타임랩스 만들기
- 타임랩스: 일정 기간의 프레임 중에서 1 프레임만 꺼내는 '빠르게 재생하기'

주어진 동영상에서 사람이 검출된 프레임을 간격을 두고 추출하여 새로운 타임랩스 영상을 만듦
- HOGDescriptor: 사람을 검출하기 위해 설정됨
- hogParrams: HOG 긱반 검출기의 파라미터 지정
-> 동영상 프레임에서 사람을 찾고 사각형으로 표시
- cv2.VideoWriter_fourcc, cv2.VideoWriter -> 새 동영상 파일 작성
    - FourCC라고 부르는 네 개의 데이터 포맷을 지정하여 동영상 파일 작성 가능
    - X, V, I, D 로 네 가지 파라미터 지정, AVI 형식으로 동영상 작성

In [ ]:
import cv2

print("타임랩스 생성 시작")

# 동영상 읽어오기 
cap = cv2.VideoCapture("mov/mov01.avi")
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# hog 선언 
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
hogParams = {'winStride': (8, 8), 'padding': (32, 32), 'scale': 1.05, 'hitThreshold':0, 'finalThreshold':5}

# 타임랩스 작성 
movie_name = "timelapse.avi"
fourcc = cv2.VideoWriter_fourcc('X', 'V', 'I', 'D')
video = cv2.VideoWriter(movie_name,fourcc, 30, (width,height))

num = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret:
        if (num%10==0):
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            human, r = hog.detectMultiScale(gray, **hogParams)
            if (len(human)>0):
                for (x, y, w, h) in human:
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (255,255,255), 3)

            video.write(frame)
    else:
        break
    num = num + 1
video.release()
cap.release()
cv2.destroyAllWindows()
print("타임랩스 생성 완료")


## 88: 전체 모습을 그래프로 가시화하기

### 사람 검출 결과를 데이터 프레임에 저장

In [ ]:
import cv2
import pandas as pd

print("분석 시작")
# 동영상 읽어오기 #
cap = cv2.VideoCapture("mov/mov01.avi")
fps = cap.get(cv2.CAP_PROP_FPS)

# hog 선언 
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
hogParams = {'winStride': (8, 8), 'padding': (32, 32), 'scale': 1.05, 'hitThreshold':0, 'finalThreshold':5}

num = 0
list_df = pd.DataFrame( columns=['time','people'] )
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret:
        if (num%10==0):
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            human, r = hog.detectMultiScale(gray, **hogParams)
            if (len(human)>0):
                for (x, y, w, h) in human:
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (255,255,255), 3)
            tmp_se = pd.Series( [num/fps,len(human) ], index=list_df.columns )
            list_df = list_df.append( tmp_se, ignore_index=True )       
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    else:
        break
    num = num + 1
cap.release()
cv2.destroyAllWindows()
print("분석 종료")

1. 동영상 파일 읽기 및 FPS 설정, 시간을 프레임에 따라 계산
2. HOG 검출기 선언으로 사람을 검출하고, 특정 파라미터로 사람을 찾도록 설정
3. 프레임 반복
4. pd.Series를 통해 각 시간에 검출된 사람의 수를 df에 추가
5. 결과 저장

### 그래프 그리기

In [ ]:
import matplotlib.pyplot as plt
plt.plot(list_df["time"], list_df["people"])
plt.xlabel('time(sec.)')
plt.ylabel('population')
plt.ylim(0,15)
plt.show()

## 89: 거리의 변화를 그래프로 확인하기

In [ ]:
import cv2
import pandas as pd

print("분석 시작")
# 동영상 읽어오기 
cap = cv2.VideoCapture("mov/mov02.avi")
fps = cap.get(cv2.CAP_PROP_FPS)

# hog 선언 
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
hogParams = {'winStride': (8, 8), 'padding': (32, 32), 'scale': 1.05, 'hitThreshold':0, 'finalThreshold':5}

num = 0
list_df2 = pd.DataFrame( columns=['time','people'] )
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret:
        if (num%10==0):
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            human, r = hog.detectMultiScale(gray, **hogParams)
            if (len(human)>0):
                for (x, y, w, h) in human:
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (255,255,255), 3)
            tmp_se = pd.Series( [num/fps,len(human) ], index=list_df.columns )
            list_df2 = list_df2.append( tmp_se, ignore_index=True )       
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    else:
        break
    num = num + 1
cap.release()
cv2.destroyAllWindows()
print("분석 종료")

In [ ]:
import matplotlib.pyplot as plt
plt.plot(list_df2["time"], list_df2["people"])
plt.xlabel('time(sec.)')
plt.ylabel('population')
plt.ylim(0,15)
plt.show()

## 90: 이동 평균을 계산해서 노이즈 제거하기
- 노이즈: 계산할 사람을 계산하지 않아서 생기는 오차, 계산하지 않아도 될 것을 계산해서 생기는 오차

In [ ]:
import numpy as np
def moving_average(x, y):
    y_conv = np.convolve(y, np.ones(5)/float(5), mode='valid')
    x_dat = np.linspace(np.min(x), np.max(x), np.size(y_conv))
    return x_dat, y_conv

In [ ]:
plt.plot(list_df["time"], list_df["people"], label="raw")
ma_x, ma_y = moving_average(list_df["time"], list_df["people"])
plt.plot(ma_x,ma_y, label="average")
plt.xlabel('time(sec.)')
plt.ylabel('population')
plt.ylim(0,15)
plt.legend()
plt.show()

In [ ]:
plt.plot(list_df2["time"], list_df2["people"], label="raw")
ma_x2, ma_y2 = moving_average(list_df2["time"], list_df2["people"])
plt.plot(ma_x2,ma_y2, label="average")
plt.xlabel('time(sec.)')
plt.ylabel('population')
plt.ylim(0,15)
plt.legend()
plt.show()

In [ ]:
plt.plot(ma_x,ma_y, label="1st")
plt.plot(ma_x2,ma_y2, label="2nd")
plt.xlabel('time(sec.)')
plt.ylabel('population')
plt.ylim(0,15)
plt.legend()
plt.show()